**Deploying SQL Server 2019 in Kubernetes** - SQLServer2019CU4.yaml defines in code a SQL Server deployment, configuration storage and service.

In [ ]:
kubectl apply -f https://raw.githubusercontent.com/nocentino/Presentations/master/DataExposed/SQLServer2019CU4.yaml

**Connect to the Instance in Azure Kubernetes Service** - In Kubernetes a Service is the persistent access point into an application. 

In [ ]:
kubectl get service
$SERVICEIP=kubectl get service mssql-deployment -o jsonpath='{.status.loadBalancer.ingress[].ip}'
$PORT=31433
Write-Output $SERVICEIP $PORT

sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SELECT @@VERSION" -P 'S0methingS@Str0ng!' 

**Crashing SQL Server** - Kubernetes will restart the crashed container in the Pod. Do this too many times too quickly it will start to back off...initially 15s. Recovery will take only a few seconds.


In [ ]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SHUTDOWN WITH NOWAIT" -P 'S0methingS@Str0ng!'

**Connect to the instance again to see if its online**

In [ ]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SELECT @@VERSION" -P 'S0methingS@Str0ng!' 

**Delete SQL Server** - Since the Deployment is defined in code...if the Pod running in the Deployment is deleted, Kubernetes will create a new one with the same configuration and storage. 

In [ ]:
$PODNAME=kubectl get pods -l app=mssql -o jsonpath='{ .items[0].metadata.name }'
kubectl delete pod $PODNAME

In [ ]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SELECT @@VERSION" -P 'S0methingS@Str0ng!' 

**Updating SQL Server** - Let's update SQL Server from 2019 CU4 to CU5 by running one line of code. In the deployment manifest, the container image is using CU5. Apply that configuration to update SQL Server.

In [ ]:
kubectl apply -f https://raw.githubusercontent.com/nocentino/Presentations/master/DataExposed/SQLServer2019CU5.yaml

In [ ]:
sqlcmd -S $SERVICEIP,$PORT -U sa -Q "SELECT @@VERSION" -P 'S0methingS@Str0ng!' 